# ERNIE模型初尝试

## 项目介绍

项目目的：使用飞桨已有的官方套件，初步实现一个粗略的相似问题回复功能。

# 数据介绍
选择飞桨自带的baike_qa2019 百科类问答json版数据集（链接：https://aistudio.baidu.com/aistudio/competition/detail/63/0/task-definition）
数据集为压缩文件，解压后有2个子文件。训练数据集baike_qa_train.json和测试数据集baike_qa_valid.json。

每条训练数据是一个字典形式，key有5个：qid问答id，category问答划分的类别(如教育/科学-理工学科-地球科学)，title问题，desc问题描述，answer回答，如下所示：
{'qid': 'qid_5982723620932473219', 'category': '教育/科学-理工学科-地球科学', 'title': '人站在地球上为什么没有头朝下的感觉 ', 'desc': '', 'answer': '地球上重力作用一直是指向球心的，因此\r\n只要头远离球心，人们就回感到头朝上。'} 

# 模型介绍
计划：通过计算文本相似性来进行问题匹配，返回和输入最大相似的问题的回复

由于：目前对飞桨了解不够充分，没有找到合适的模型。通过在项目集搜索，找到了使用paddlehub的ERNIE-GEN进行问答系统搭建的项目。

ERNIE的github网址为 https://github.com/PaddlePaddle/ERNIE ，里面介绍：ERNIE在工业界得到了大规模应用，如搜索引擎、新闻推荐、广告系统、语音交互、智能客服等。
而2020.5开源的 ERNIE-GEN 模型是最强文本生成预训练模型。

所以ernie-gen可能更偏向于文本生成，非返回原有的资源库结果。但是目前没有发现可用的官方套件，所以想用ernie-gen尝试一下。

In [1]:
# 1.1 查看工作区文件, 该目录下的变更将会持久保存. 请及时清理不必要的文件, 避免加载过慢.
# !ls /home/aistudio/work

# 1.2 解压挂载的数据集在同级目录下
!unzip -oq data/data107726/百科类问答json版.zip -d data/data107726/

# 1.3 查看当前挂载的数据集目录, 该目录下的变更重启环境后会自动还原
!ls /home/aistudio/data/data107726

baike_qa_train.json  baike_qa_valid.json  百科类问答json版.zip


In [ ]:
# 1.4 持久化安装, 需要使用持久化路径
# !mkdir /home/aistudio/external-libraries
# !pip install paddle-ernie -t /home/aistudio/external-libraries
# !pip install paddlehub==2.1.1 -t /home/aistudio/external-libraries

# 1.5 持久化安装后，添加如下代码, 这样每次环境(kernel)启动的时候只要运行下方代码即可: 
# import sys 
# sys.path.append('/home/aistudio/external-libraries')

In [2]:
!pip install paddlehub==2.1.1
!pip install paddle-ernie

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     |████████████████████████████████| 211 kB 7.6 MB/s            
     |████████████████████████████████| 95 kB 7.2 MB/s             
     |████████████████████████████████| 12.2 MB 4.7 MB/s            
  Attempting uninstall: paddlehub
    Found existing installation: paddlehub 2.0.4
    Uninstalling paddlehub-2.0.4:
      Successfully uninstalled paddlehub-2.0.4
You should consider upgrading via the '/opt/conda/envs/python35-paddle120-env/bin/python -m pip install --upgrade pip' command.
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Preparing metadata (setup.py) ... done
  Created wheel for paddle-ernie: filename=paddle_ernie-0.2.0.dev1-py3-none-any.whl size=25548 sha256=d4b5c4d249b94d519c336c239d6b02540ff195f22f19e7161aa04939b5199c93
  Stored in directory: /home/aistudio/.cache/pip/wheels/cf/b5/ea/1dc4bdb751cd206d020538bf525ff58d5a52fc4e0df4531123
Successfully built paddle-ernie
You should consider upgra

# 模型训练
## 数据处理
ERNIE-GEN模型要求的数据输入格式是：编号\t文本1\t文本2的格式，和原数据格式不符合，故先对数据进行处理

In [3]:
# 2. 处理数据格式
# 根据模型要求，处理成：编号\t问题\t答案的形式
import json

def resave_2txt(raw_path, raw_encod, save_path, save_encod, max_len=None):
    try:
        max_len = int(max_len)
    except:
        max_len = float('inf')
    tmp_len = 0
    save_txt = []
    with open(raw_path,'r',encoding=raw_encod) as f:
        for line in f.readlines():
            dic_v = json.loads(line)
            txt_line = dic_v['qid'] + '\t' + dic_v['title'].replace('\n','').replace('\r','')  + '\t' + dic_v['answer'].replace('\n','').replace('\r','')
            if len(txt_line)>500:
                pass
            else:
                if tmp_len <= max_len:
                    save_txt.append(txt_line)
                    tmp_len += 1
                    with open(save_path, 'a+', encoding=save_encod) as f:
                        f.write(txt_line + '\n')
                else:
                    break
    
    return save_txt


train_txt = resave_2txt('data/data107726/baike_qa_train.json', 'utf-8', 'data/data107726/baike_qa_train.txt', 'utf-8', 5000)
valid_txt = resave_2txt('data/data107726/baike_qa_valid.json', 'utf-8', 'data/data107726/baike_qa_valid.txt', 'utf-8')
print(f'训练集数据量：{len(train_txt)}, 验证集数据量：{len(valid_txt)}\n')
print('训练集数据样例：\n', train_txt[0])

训练集数据量：5001, 验证集数据量：39908

训练集数据样例：
 qid_5982723620932473219	人站在地球上为什么没有头朝下的感觉 	地球上重力作用一直是指向球心的，因此只要头远离球心，人们就回感到头朝上。


## 模型训练
由于对自己的要求是使用已有套件，初步实现目的。除了报错之外，对于参数暂时没有太多研究。

训练时发现的报错有：
1. 数据集的处理：一些问题回复本身含\n，需要替换成其他字符，否则保存数据的时候，\n之后的数据单独为一条记录，不符合要求的数据格式(编号\t文本\t文本)
2. max_encode_len(int)最长编码长度 和  max_decode_len(int) 最长解码长度，受输入的数据长度影响，若数据长度较长，而此处设置的比较小，则会报错
3. batch_size：有一些报错会提出需要调小batch_size


In [4]:
# 3.训练模型
# 暂时无脑先走流程，流程走完再调具体
# 具体流程和参数可参考链接：https://aistudio.baidu.com/aistudio/projectdetail/1456984
# 大神链接(可能是版本问题，运行报错)：

import paddlehub as hub
module = hub.Module(name="ernie_gen")

result = module.finetune(
    use_gpu = True,
    train_path='data/data107726/baike_qa_train.txt',
    # dev_path='data/data107726/baike_qa_valid.txt',
    save_dir="ernie_gen_result",
    max_steps=200,
    max_encode_len=1024,
    max_decode_len=1024,
    noise_prob=0.2,
    batch_size=2,
    log_interval=20
)

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/__init__.py:107: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import MutableMapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/rcsetup.py:20: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable, Mapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/colors.py:53: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sized


Download https://bj.bcebos.com/paddlehub/paddlehub_dev/ernie_gen_1.1.0.tar.gz
[##################################################] 100.00%
Decompress /home/aistudio/.paddlehub/tmp/tmpf22fac_k/ernie_gen_1.1.0.tar.gz
[##################################################] 100.00%


[2022-02-25 18:59:56,338] [    INFO] - Successfully installed ernie_gen-1.1.0
[2022-02-25 18:59:56,340] [    INFO] - Downloading https://paddlenlp.bj.bcebos.com/models/transformers/ernie/vocab.txt and saved to /home/aistudio/.paddlenlp/models/ernie-1.0
[2022-02-25 18:59:56,343] [    INFO] - Downloading vocab.txt from https://paddlenlp.bj.bcebos.com/models/transformers/ernie/vocab.txt
100%|██████████| 90/90 [00:00<00:00, 2408.92it/s]
[2022-02-25 18:59:56,509] [    INFO] - Downloading https://paddlenlp.bj.bcebos.com/models/transformers/ernie/ernie_v1_chn_base.pdparams and saved to /home/aistudio/.paddlenlp/models/ernie-1.0
100%|██████████| 392507/392507 [00:12<00:00, 32667.66it/s]
[2022-02-25 19:00:08,652] [   DEBUG] - init ErnieModel with config: {'attention_probs_dropout_prob': 0.1, 'hidden_act': 'relu', 'hidden_dropout_prob': 0.1, 'hidden_size': 768, 'initializer_range': 0.02, 'max_position_embeddings': 513, 'num_attention_heads': 12, 'num_hidden_layers': 12, 'type_vocab_size': 2, 'vo

In [5]:
# 查看训练结束时的模型保存路径 和 模型困惑度
print(result)

{'last_save_path': 'ernie_gen_result/step_200_ppl_127509.62500.params', 'last_ppl': 127509.625}


In [6]:
# 导出模型
module.export(params_path=result['last_save_path'], module_name="ernie_gen_baikeqa", author="zhuqianer")

[2022-02-25 19:00:58,325] [    INFO] - Begin export the model save in ernie_gen_result/step_200_ppl_127509.62500.params ...
[2022-02-25 19:00:59,149] [    INFO] - The module has exported to /home/aistudio/ernie_gen_baikeqa


In [7]:
# 安装module
!hub install ernie_gen_baikeqa

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/__init__.py:107: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import MutableMapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/rcsetup.py:20: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable, Mapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/colors.py:53: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sized
[2022-02-25 19:01:07,336] [    INFO] - Successfully installed ernie_gen_baikeqa-1.0.0


In [9]:
print(valid_txt[0])

qid_1815059893214501395	请问深入骨髓地喜欢一个人怎么办我不能确定对方是不是喜欢我，我却想 	一定要告诉他你很喜欢他 很爱他!!  虽然不知道你和他现在的关系是什么！但如果真的觉得很喜欢就向他表白啊！！起码你努力过了！  女生主动多少占一点优势的！！呵呵  只愿曾经拥有！  到以后就算感情没现在这么强烈了也不会觉得遗憾啊~！  与其每天那么痛苦的想他 恋他 还不如直接告诉他 ！  不要怕回破坏你们现有的感情！因为如果不告诉他  你可能回后悔一辈子！！  


## 模型预测
可以看到，下面的预测结果什么都不是，无论输入什么，返回的都是这句话。

可能是训练不足，或者是没有加入验证集进行训练？但是就算我输入训练集的部分字段还是同样的返回结果。重新进行训练又会报内存不足/建议减小batch_size，所以这是我目前唯一能拿的出手的东西了，虽然返回值莫名奇妙，但至少预测的时候没报错。

In [11]:
# 测试

import paddlehub as hub

module = hub.Module(name="ernie_gen_baikeqa")

test_texts = ['"我爱你"古文怎么说?']
# generate包含3个参数，texts为输入文本列表，use_gpu指定是否使用gpu，beam_width指定beam search宽度。
results = module.generate(texts=test_texts, use_gpu=False, beam_width=5)
for result in results:
    print(result)

[2022-02-25 19:02:08,520] [    INFO] - Already cached /home/aistudio/.paddlenlp/models/ernie-1.0/ernie_v1_chn_base.pdparams
[2022-02-25 19:02:08,522] [   DEBUG] - init ErnieModel with config: {'attention_probs_dropout_prob': 0.1, 'hidden_act': 'relu', 'hidden_dropout_prob': 0.1, 'hidden_size': 768, 'initializer_range': 0.02, 'max_position_embeddings': 513, 'num_attention_heads': 12, 'num_hidden_layers': 12, 'type_vocab_size': 2, 'vocab_size': 18000, 'pad_token_id': 0}
[2022-02-25 19:02:11,954] [    INFO] - loading pretrained model from /home/aistudio/.paddlenlp/models/ernie-1.0/ernie_v1_chn_base.pdparams
[2022-02-25 19:02:12,793] [    INFO] - param:mlm_bias not set in pretrained model, skip
[2022-02-25 19:02:12,796] [    INFO] - param:mlm.weight not set in pretrained model, skip
[2022-02-25 19:02:12,797] [    INFO] - param:mlm.bias not set in pretrained model, skip
[2022-02-25 19:02:12,799] [    INFO] - param:mlm_ln.weight not set in pretrained model, skip
[2022-02-25 19:02:12,801] [  

['你是你你你你你的你你你你你你你你你你的的的的的的的的的的的的的的的的的的；；；；；；；；；；；；；；', '你是你你你你你的你你你你你你你你你你的的的的的的的的的的的的的的的的的的的的；；；；；；；；；；；；', '你是你你你你你的你你你你你你你你你你的的的的的的的的的的的的的的的的的的的；；；；；；；；；；；；；', '你是你你你你你的你你你你你你你你你你你的的的的的的的的的的的的的的的的的的的；；；；；；；；；；；；', '你是你你你你你的你你你你你你你你你你的的的的的的的的的的的的的的的的的的；；；；；；；；；；；；；的']


## 个人总结
在算法程序不断迭代的时代，没赶上时代的步伐，就不要走捷径。

原本想要靠已有的套件和大神共享的脚本，直接套用得到一个初步结果，再进行后续的调优，但是结果一塌糊涂。
首先是模型选择的问题，自己初步设想是返回相似度最高的问题的已有答案，但是选择了一个文本生成的模型（因为看到有人用这个去做，想走捷径）。
然后是搜到的项目不能完全复刻结果，运行的时候报错，可能是软件包版本的问题。

总结还是要了解模型之后，才能更好的运用它。

好处是我对这个ERNIE模型产生了兴趣，后续一定要运行一个可执行的脚本来体验一下



# 提交链接
aistudio链接：https://aistudio.baidu.com/aistudio/projectdetail/3521598?contributionType=1

github链接：

gitee链接

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 